In [ ]:
%load_ext autoreload 
%autoreload 2

# NOTE: 
I ended up finidng issues in the data where downsampling was down incorrectly, this notebook is only saved in the interm. LSTM 01 and 02 are invalid and have been deleted. This notebook is only here for reference

# Initial exploration of full training LSTM simple 01


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

from src.modeling.prototyping.lstm_protype_simple import validated_tf
from src.modeling.prototyping.lstm_simple_initial_training import build_path

### Tensorflow spec

In [ ]:
validated_tf()

### model specs

In [ ]:
LTSM_1_PATH = '/projects/p31961/dopamine_modeling/results/models/lstm_simple_initial_training_with_downsampling'

In [ ]:
ltsm = tf.keras.models.load_model(LTSM_1_PATH)

In [ ]:
ltsm.summary()

* data was downsampled during preprocessing selecting every 100th data point

In [ ]:
PATH_TO_DATA = '/projects/p31961/gaby_data/aggregated_data/data_pipeline/datasets'


### reformating and data aggregation


In [ ]:
def combine_data(df, true_signal, infered_signal):
    cols_to_group = ['day', 'time', 'event_cue', 'event_shock', 'sensor_D1', 'sensor_D2', 'sensor_DA']
    agg_diict = {'true_signal': ['mean', 'sem'],
             'predicted_signal': ['mean', 'sem']
             }
    def flatten_df(df):
        df.columns = ['_'.join(col) for col in df.columns.values]
        df = df.reset_index()
        return df
    def strip_cols(df):
        df.columns = [col.rstrip('_') for col in df.columns]
        return df
    return(
        df.assign(
            true_signal=true_signal, 
            predicted_signal=infered_signal)
        .reset_index(drop=True)
        .groupby(by = cols_to_group, as_index = False).agg(agg_diict)
        .pipe(flatten_df)
        .pipe(strip_cols)
        .drop(columns = 'index')

    )

# Summary 
the data is clearly incorrect. I think data is downsampled too much, I have preprocessed the data gain sampling every 10th data point and retrained to LSTM simple 02

_____

# Validation of LSTM_02_simple retran with smaller downsample (25)


In [111]:
def plot_signal(y):
    sns.lineplot(data=day6_cue_da, x='time', y=f'{y}_mean', color='black', linewidth=0.5)
    plt.fill_between(day6_cue_da['time'], day6_cue_da[f'{y}_mean'] - day6_cue_da[f'{y}_sem'], day6_cue_da[f'{y}_mean'] + day6_cue_da[f'{y}_sem'], alpha=.2, color='black')
plot_signal('true_signal')
plot_signal('predicted_signal')

NameError: name 'day6_cue_da' is not defined

# what is wrong with the data?`

In [ ]:
RAW_PATH = '/projects/p31961/gaby_data/aggregated_data/aggregated_data.parquet.gzp'
raw_data = pd.read_parquet(RAW_PATH)
raw_data

In [ ]:
raw_ds25 = raw_data[::100]
raw_ds25

In [ ]:
def filter_data(df):
    def flatten_df(df):
        df.columns = ['_'.join(col) for col in df.columns.values]
        df = df.reset_index()
        return df
    def strip_cols(df):
        df.columns = [col.rstrip('_') for col in df.columns]
        return df
        
    group_by = ['time', 'sensor', 'trial', 'mouse_id', 'day', 'event']
    agg_dict = {'signal': ['mean']}

    grouped= (
        df.groupby(by=group_by, as_index=False).agg(agg_dict)
        .pipe(flatten_df)
        .pipe(strip_cols)
        .drop(columns = 'index')
        .sort_values(by = [col for col in group_by if col != 'time'])
        [::100]
    )
    return (grouped
            # .query('day==1 & event=="cue" & sensor=="DA"')
            # .groupby(by = ['time'], as_index=True).agg({'signal_mean': ['mean', 'sem']})
            # .pipe(flatten_df)
            # .pipe(strip_cols)
    )

d1_cue_da_ds = filter_data(raw_ds25)


In [ ]:
raw_filter_ds = filter_data(raw_data)

In [ ]:
raw_filter_ds.query("mouse_id=='310_907' & event== 'cue' & sensor=='DA' & day==3")

In [ ]:
raw_data.query("mouse_id=='310_907' & event== 'cue' & sensor=='DA' & day==3")

In [ ]:
nan_vals = raw_data[raw_data.isna().any(axis=1)]

# Missing values from signal?

In [ ]:
group_by = ['mouse_id', 'trial',  'day', 'event', 'sensor']
nan_vals.groupby(by = group_by, as_index=False).count().drop(columns = ['time', 'signal'])

In [ ]:
path_to_save = f'/projects/p31961/gaby_data/aggregated_data/aggregated_data'
new_path = os.path.join(path_to_save, 'aggregated_data_test_downsample')
raw_filter_ds.to_parquet(new_path)

In [ ]:
d1_cue_da_ds.sort_values(by=['sensor', 'trial', 'mouse_id', 'day', 'event'])

In [ ]:
d1_cue_da_raw = filter_data(raw_data)

In [ ]:
d1_cue_da_raw
# correct_down_sample = d1_cue_da_raw[::100]

In [112]:
def plot_data(df, title):
    sns.lineplot(data=df, x='time', y='signal_mean_mean',linewidth=0.2)
    # change y axsis to be -2 to 2
    plt.fill_between(df['time'], df['signal_mean_mean'] - df['signal_mean_sem'], df['signal_mean_mean'] + df['signal_mean_sem'], alpha=.2, color='black')
    plt.ylim(-2, 2)
    plt.title(title)
    
# plot_data(df = d1_cue_da_raw, title='raw data')

In [113]:

plot_data(df = d1_cue_da_ds, title='downsampled data')


ValueError: Could not interpret value `signal_mean_mean` for parameter `y`

In [ ]:
plot_data(df = correct_down_sample, title='correct downsampled data')

In [ ]:
updated_ds_query = .query('day==1 & event=="cue" & sensor=="DA"')

# error in down sampling
* I made a mistake in when I introduced the downsampling for this data. I think I need to introduce it after grouping and processing right before training? The below is an interative process for finding the correct order of operations

In [119]:
def filter_data_update(df):
    def flatten_df(df):
        df.columns = ['_'.join(col) for col in df.columns.values]
        df = df.reset_index()
        return df
    def strip_cols(df):
            df.columns = [col.rstrip('_') for col in df.columns]
            return df
        
    group_by = ['time', 'sensor', 'day', 'event']
    agg_dict = {'signal': ['mean', 'sem']}

    grouped= (
        df.groupby(by=group_by, as_index=False).agg(agg_dict)
        .pipe(flatten_df)
        .pipe(strip_cols)
        .drop(columns = 'index') 
    )
    return (grouped
            # .query('day==1 & event=="cue" & sensor=="DA"')
            # .groupby(by = ['time', ], as_index=True).agg({'signal_mean': ['mean', 'sem']})
            # .pipe(flatten_df)
            # .pipe(strip_cols)
    )


grouped_data_update = filter_data_update(raw_data)

In [ ]:
day1_cue_da = grouped_data_update.query('day==1 & event=="cue" & sensor=="DA"')
day1_cue_da

In [ ]:
def plot_data_update(df, title):
    sns.lineplot(data=df, x='time', y='signal_mean',linewidth=0.2)
    # change y axsis to be -2 to 2
    plt.fill_between(df['time'], df['signal_mean'] - df['signal_sem'], df['signal_mean'] + df['signal_sem'], alpha=.2, color='black')
    plt.ylim(-2, 2)
    plt.title(title)
plot_data_update(df = day1_cue_da, title='raw data updated filter')

## I beleive the downsampling after the aggregated and melted dataframe is the problem. 
* possible soluton - convert time to timeseries in padas -> resample 


In [ ]:
raw_data

In [ ]:
agg_dict = {'mouse_id':'first', 
            'day':'first',
            'event': 'first',
            'trial': 'first', 
            'sensor': 'first', 
            'signal': 'mean'}

In [ ]:
raw_data_us = raw_data.assign(time =pd.to_timedelta(raw_data['time'], unit='seconds'))
resample = raw_data_us.resample('100ms', on='time').agg(agg_dict)
resample

In [ ]:

def agg_data(df):

        def flatten_df(df):
            df.columns = ['_'.join(col) for col in df.columns.values]
        #     df = df.reset_index()
            return df
        return (
                df
                # .assign(time =pd.to_timedelta(df['time'], unit='seconds'))
                # .set_index('time')
                .groupby(by = ['mouse_id', 'day', 'time', 'event', 'trial', 'sensor'], as_index=False)
                # .resample('100ms')
                .mean()
                # .pipe(flatten_df)
                )
resample = agg_data(raw_ds25)
resample

In [ ]:
raw_ds25.sort_values(by=['mouse_id', 'day', 'event', 'trial', 'time', 'sensor'])

In [ ]:
def sort_and_ds(df):
    return (
        df
        .sort_values(by=['mouse_id', 'day', 'event', 'trial', 'sensor'])
        # .reset_index(drop=True)
        [::2]
    )
sort_and_ds(raw_ds25)

In [118]:
d1_cue_da_ds = grouped_data_update.query('day==1 & event=="cue" & sensor=="DA"')

NameError: name 'grouped_data_update' is not defined

In [117]:
d1_cue_da_ds.group_by()

,time,sensor,trial,mouse_id,day,event,signal_mean
454210,-24.931187,DA,0,142_237,1,cue,0.198750
1733710,-24.734577,DA,0,142_237,1,cue,1.011394
3013210,-24.537968,DA,0,142_237,1,cue,0.345653
4292710,-24.341358,DA,0,142_237,1,cue,0.950242
5572210,-24.144749,DA,0,142_237,1,cue,-1.777425
...,...,...,...,...,...,...,...
287631594,19.195868,DA,28,313_255,1,cue,0.079195
288911094,19.392477,DA,28,313_255,1,cue,-0.590885
290190594,19.589087,DA,28,313_255,1,cue,-0.307998
291470094,19.785696,DA,28,313_255,1,cue,-0.901103
